In [81]:

import cirq
from qiskit import QuantumCircuit
# Define qubits
q0, q1 = cirq.LineQubit.range(2)

# Build circuit
circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.CNOT(q0, q1),
    cirq.measure(q0, q1, key='c')
)

# Export to QASM
qasm_output = cirq.qasm(circuit)

# ---- Cleanup: remove comments + blank lines + rename creg ----
lines = []
for line in qasm_output.splitlines():
    line = line.strip()              # remove trailing spaces
    if not line:                     # skip empty lines
        continue
    if line.startswith("//"):        # skip comments
        continue
    line = line.replace("m_c", "c")  # rename classical register
    lines.append(line)

qasm_clean = "\n".join(lines)
print(qasm_clean)


OPENQASM 2.0;
include "qelib1.inc";
qreg q[2];
creg c[2];
h q[0];
cx q[0],q[1];
measure q[0] -> c[0];
measure q[1] -> c[1];


In [82]:
# Get QASM string
qasm_str = cirq.qasm(circuit)

# Save to file
with open("myfile.qasm", "w") as f:
    f.write(qasm_str)

In [83]:

loaded_qiskit_circuit = QuantumCircuit.from_qasm_file("myfile.qasm")
print("Reloaded circuit from QASM (Qiskit):")
print(loaded_qiskit_circuit)

Reloaded circuit from QASM (Qiskit):
       ┌───┐     ┌─┐   
  q_0: ┤ H ├──■──┤M├───
       └───┘┌─┴─┐└╥┘┌─┐
  q_1: ─────┤ X ├─╫─┤M├
            └───┘ ║ └╥┘
m_c: 2/═══════════╩══╩═
                  0  1 


In [84]:
# Remove Qiskit and qasm2
#import qiskit
#import qiskit.qasm2
import numpy as np
import requests
import os
import json


# current Quokka address:
req_str_qasm = 'http://quokka3.quokkacomputing.com/qsim/qasm'

In [85]:

# Build circuit
q0, q1 = cirq.LineQubit.range(2)
circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.CNOT(q0, q1),
    cirq.measure(q0, q1, key='c')
)

# Export circuit to JSON
circuit_json = cirq.to_json(circuit)

# Optional: remove metadata or extra fields if Quokka complains
# For example, keep only gates and qubits
circuit_dict = json.loads(circuit_json)
simplified_circuit_json = json.dumps({
    "schema": circuit_dict.get("schema"),
    "moments": circuit_dict.get("moments"),
    "qubits": circuit_dict.get("qubits")
})

print(simplified_circuit_json)


{"schema": null, "moments": [{"cirq_type": "Moment", "operations": [{"cirq_type": "GateOperation", "gate": {"cirq_type": "HPowGate", "exponent": 1.0, "global_shift": 0.0}, "qubits": [{"cirq_type": "LineQubit", "x": 0}]}]}, {"cirq_type": "Moment", "operations": [{"cirq_type": "GateOperation", "gate": {"cirq_type": "CXPowGate", "exponent": 1.0, "global_shift": 0.0}, "qubits": [{"cirq_type": "LineQubit", "x": 0}, {"cirq_type": "LineQubit", "x": 1}]}]}, {"cirq_type": "Moment", "operations": [{"cirq_type": "GateOperation", "gate": {"cirq_type": "MeasurementGate", "num_qubits": 2, "key": "c", "invert_mask": []}, "qubits": [{"cirq_type": "LineQubit", "x": 0}, {"cirq_type": "LineQubit", "x": 1}]}]}], "qubits": null}


In [86]:

# --- Prepare request payload ---
data = {
    'script': simplified_circuit_json,  # send JSON instead of QASM string
    'count': 10
    }

# --- Send to Quokka ---
req_str_qasm = 'http://quokka3.quokkacomputing.com/qsim/qasm'
response = requests.post(req_str_qasm, json=data)

# --- Parse response ---
json_obj = response.json()
print(json.dumps(json_obj, indent=2))

{
  "error": "Syntax error while parsing token \"{\" in script at line 1",
  "error_code": 7,
  "result": false
}


In [87]:
json_obj['result'] = [[True, False], [False, True], ...]
bits = [int(b) for shot in json_obj['result'] for b in shot]

print(bits)

TypeError: 'ellipsis' object is not iterable

In [ ]:
%pip install randtest
import randtest

randtest.random_score(bits)

Note: you may need to restart the kernel to use updated packages.


NameError: name 'bits' is not defined